In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
train_file = "./train.csv"
test_file = "./test.csv"
num_cols = ["ps_reg_01", "ps_reg_02", "ps_reg_03","ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"]
ignore_cols = ["id", "target", "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04", "ps_calc_05", 
               "ps_calc_06", "ps_calc_07", "ps_calc_08", "ps_calc_09", "ps_calc_10", "ps_calc_11", 
               "ps_calc_12", "ps_calc_13", "ps_calc_14","ps_calc_15_bin", "ps_calc_16_bin", 
               "ps_calc_17_bin","ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"]

In [3]:
def overview():    
    dfTrain = pd.read_csv(train_file)
    dfTest = pd.read_csv(test_file)
    df = pd.concat([dfTrain,dfTest], sort=False)

    field_size = len(df.columns) - len(ignore_cols)
    feature_dict = {}
    feature_size = 0
    for col in df.columns:
        if col in ignore_cols:
            continue
        elif col in num_cols:
            feature_dict[col] = feature_size
            feature_size += 1
        else:
            unique_val = df[col].unique()
            feature_dict[col] = dict(zip(unique_val,range(feature_size,len(unique_val) + feature_size)))
            feature_size += len(unique_val)
    return dfTrain, field_size, feature_size, feature_dict

In [4]:
def preprocess(train_df, split):
    label_df = train_df[['target']]
    train_df.drop(['target','id'],axis=1,inplace=True)
    feature_idx = train_df.copy()
    feature_val = train_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1        
            
    split_idx = feature_idx.shape[0] - round(feature_idx.shape[0]*split)
    train_input = [feature_idx[:split_idx].values, feature_val[:split_idx].values]
    train_y = label_df[:split_idx].values
    validate_input = [feature_idx[split_idx:].values, feature_val[split_idx:].values]
    validate_y = label_df[split_idx:].values

    return train_input, train_y, validate_input, validate_y

In [5]:
dfTrain, field_size, feature_size, feature_dict = overview()

In [6]:
train_input, train_y, validate_input, validate_y = preprocess(dfTrain, 0.1)

In [7]:
class DeepFM(tf.keras.Model):
    def __init__(self, cfg):
        super(DeepFM, self).__init__()
        self.feature_size = cfg['feature_size']
        self.field_size = cfg['field_size']
        self.embed_size = cfg['embed_size']
        self.deep_nn = cfg['deep_nn']
        
        self.dropout_fm = cfg['dropout_fm']
        self.dropout_deep = cfg['dropout_deep']
        
        # fm        
        self.feature_weight = tf.keras.layers.Embedding(cfg['feature_size'], 1)
        self.feature_embed = tf.keras.layers.Embedding(cfg['feature_size'], cfg['embed_size'])

        # dnn
        for layer in range(len(cfg['deep_nn'])):
            setattr(self, 'dense_' + str(layer), tf.keras.layers.Dense(self.deep_nn[layer]))
            setattr(self, 'batchNorm_' + str(layer), tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(layer), tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(layer), tf.keras.layers.Dropout(self.dropout_deep))
            
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=True)

    def call(self, inputs, training=False):
        # inputs = [feature_idx, feature_val]
        reshaped_feature_val = tf.cast(tf.reshape(inputs[1], shape=[-1,self.field_size,1]), tf.float32)
        # linear        
        weights = self.feature_weight(inputs[0])
        linear = tf.reduce_sum(tf.multiply(weights,reshaped_feature_val),2)
        
        # fm  
        embeddings = self.feature_embed(inputs[0])
        second_inner = tf.multiply(embeddings,reshaped_feature_val)
        
        summed_features_emb = tf.reduce_sum(second_inner,1)
        summed_features_emb_square = tf.square(summed_features_emb)
        
        squared_features_emb = tf.square(second_inner)
        squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)
        
        fm = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)
        
        # dnn
        y_deep = tf.reshape(embeddings,shape=[-1,self.field_size * self.embed_size])
        for layer in range(0, len(self.deep_nn)):
            y_deep = getattr(self, 'dense_' + str(layer))(y_deep)
            y_deep = getattr(self, 'batchNorm_' + str(layer))(y_deep)
            y_deep = getattr(self, 'activation_' + str(layer))(y_deep)
            y_deep = getattr(self, 'dropout_' + str(layer))(y_deep)
            
        # concat
        concat = tf.concat([linear, fm, y_deep], axis=1)                                
        out = self.fc(concat)
        return out

In [8]:
cfg = {
    "feature_size": feature_size,
    "field_size": field_size,
    "embed_size":8,
    "deep_nn":[32,32],
    "dropout_fm": 0,
    "dropout_deep": 0.2,
    "epoch":20,
    "batch":10000
}

In [26]:
model = DeepFM(cfg)
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['binary_accuracy', 'AUC'])
model.fit(train_input, train_y, epochs=cfg['epoch'], batch_size=cfg['batch'], shuffle=True,
          verbose=1, validation_data=(validate_input, validate_y))

Train on 535691 samples, validate on 59521 samples
Epoch 1/20
535691/535691 [==============================] - 3s 5us/sample - loss: 0.3255 - binary_accuracy: 0.9190 - AUC: 0.5123 - val_loss: 0.2037 - val_binary_accuracy: 0.9633 - val_AUC: 0.4658
Epoch 2/20
535691/535691 [==============================] - 1s 2us/sample - loss: 0.1586 - binary_accuracy: 0.9636 - AUC: 0.5566 - val_loss: 0.1654 - val_binary_accuracy: 0.9633 - val_AUC: 0.5009
Epoch 3/20
535691/535691 [==============================] - 1s 2us/sample - loss: 0.1568 - binary_accuracy: 0.9636 - AUC: 0.5815 - val_loss: 0.1608 - val_binary_accuracy: 0.9633 - val_AUC: 0.5217
Epoch 4/20
535691/535691 [==============================] - 1s 2us/sample - loss: 0.1557 - binary_accuracy: 0.9636 - AUC: 0.5970 - val_loss: 0.1586 - val_binary_accuracy: 0.9633 - val_AUC: 0.5433
Epoch 5/20
535691/535691 [==============================] - 1s 2us/sample - loss: 0.1553 - binary_accuracy: 0.9636 - AUC: 0.6002 - val_loss: 0.1571 - val_binary_accu

In [10]:
def prepredict(path):
    predict_df = pd.read_csv(path)
    feature_idx = predict_df.copy()
    feature_val = predict_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1 
    return [feature_idx.values, feature_val.values]

In [13]:
predict_input = prepredict(test_file)
label_pre = model.predict(predict_input)

In [14]:
type(label_pre)

numpy.ndarray

In [19]:
label_pre

array([[0.02657855],
       [0.03356069],
       [0.03306636],
       ...,
       [0.05187541],
       [0.03323221],
       [0.0299944 ]], dtype=float32)

In [20]:
sorted_idx = np.argsort(-label_pre, axis=0)
sorted_val = label_pre[sorted_idx]

In [21]:
sorted_val

array([[[0.5062654 ]],

       [[0.5047808 ]],

       [[0.5031859 ]],

       ...,

       [[0.00576538]],

       [[0.00562268]],

       [[0.00548503]]], dtype=float32)

In [27]:
label_pre_for_train = model.predict(train_input)

In [28]:
label_pre_for_train_pd = pd.DataFrame(label_pre_for_train)
label_pre_for_train_pd.head(40)

,0
0,0.059662
1,0.025427
2,0.020657
3,0.014821
4,0.037768
5,0.038043
6,0.021303
7,0.010496
8,0.029850
9,0.059881


In [29]:
train_y_pd = pd.DataFrame(train_y)
train_y_pd.head(40)

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,1
